In [ ]:
import torch
import numpy
import torch.nn as nn
from torchvision.models import resnet50
from torch import Tensor
from typing import Dict, Iterable, Callable
import struct
import time
import re

# Change to True to generate corresponding values
write_layer_outputs_to_file = False
write_layer_params_to_file  = False

features = {}
def get_features(name):
  def hook(model, input, output):
    features[name] = output.detach()
  return hook

model = resnet50(pretrained=True)
model.eval()

inp = torch.ones(1, 3, 736, 1280)

raw_layers = []
for layer in model.named_modules():
  raw_layers.append(layer[0])

leaf_layers = []
for i in range(1, len(raw_layers)-1):
  curr_layer = raw_layers[i]
  next_layer = raw_layers[i+1]
  if(next_layer[:len(curr_layer)+1] != curr_layer + "."):
    leaf_layers.append(curr_layer)
leaf_layers.append(next_layer)

layers = []
for i in range(len(leaf_layers)):
  layers.append(re.sub(r"\.(\d)",r"[\1]",leaf_layers[i]))

for i in range(len(layers)):
  layer = layers[i]
  layer_hook = "model." + layer + ".register_forward_hook(get_features('" + layer + "'))"
  exec(layer_hook)

# Run inference
outp = model(inp)

EPS = 10 ** -5 # constant

if(write_layer_outputs_to_file):
  # Write layer outputs
  for i in range(len(layers)):
    layer = layers[i]
    if(layer in features.keys()):
      layer_name = layer.replace("].","_")
      layer_name = layer_name.replace("[", "_")
      layer_name = layer_name.replace("]", "")
      filename = "fused_resnet_bin/" +  layer_name + ".bin"
      with open(filename,"wb") as f:
        features[layer].cpu().numpy().tofile(f)
      print("Layer " + str(i) + " feature map printed to " + filename)

if(write_layer_params_to_file):
  # Write layer params
  for i in range(len(layers)):
    layer = layers[i]
    if('conv' in layer or 'downsample[0]' in layer):
      conv_layer_name = layer.replace("].","_")
      conv_layer_name = conv_layer_name.replace("[", "_")
      conv_layer_name = conv_layer_name.replace("]", "")
      
      conv_param_name = layer.replace("[",".")
      conv_param_name = conv_param_name.replace("]","")
      
      conv_weight = model.state_dict()[conv_param_name+'.weight']

    if('bn' in layer or 'downsample[1]' in layer):
      bn_layer_name = layer.replace("].","_")
      bn_layer_name = bn_layer_name.replace("[", "_")
      bn_layer_name = bn_layer_name.replace("]", "")
      
      bn_param_name = layer.replace("[",".")
      bn_param_name = bn_param_name.replace("]","")
      
      bn_weight = model.state_dict()[bn_param_name+'.weight']
      bn_bias   = model.state_dict()[bn_param_name+'.bias']
      bn_mean   = model.state_dict()[bn_param_name+'.running_mean']
      bn_var    = model.state_dict()[bn_param_name+'.running_var']

      bn_factor    = torch.div(bn_weight,torch.sqrt(bn_var+EPS)).view(-1,1,1,1)
      fused_weight = torch.mul(conv_weight, bn_factor)
      fused_bias   = bn_bias - torch.div(torch.mul(bn_weight,bn_mean),torch.sqrt(bn_var+EPS))

      if('downsample' in bn_layer_name):
        layer_number = '0'
        layer_prefix = bn_layer_name[0:bn_layer_name.find('downsample')]
      else:
        layer_number = conv_layer_name[-1]
        layer_prefix = bn_layer_name[0:bn_layer_name.find('bn')]
      
      weights_filename = "fused_resnet_bin/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_weights.bin"
      bias_filename    = "fused_resnet_bin/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_bias.bin"

      with open(weights_filename, "wb") as f:
        fused_weight.detach().numpy().tofile(f)
      print("Fused weights of " + layer_prefix + layer_number + " printed to file " + weights_filename)

      with open(bias_filename, "wb") as f:
        fused_bias.detach().numpy().tofile(f)
      print("Fused biases of " + layer_prefix + layer_number + " printed to file " + bias_filename)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 151MB/s]


In [ ]:
!git clone https://github.com/Tianxiaomo/pytorch-YOLOv4.git

Cloning into 'pytorch-YOLOv4'...
remote: Enumerating objects: 1049, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1049 (delta 2), reused 0 (delta 0), pack-reused 1043
Receiving objects: 100% (1049/1049), 2.39 MiB | 16.42 MiB/s, done.
Resolving deltas: 100% (644/644), done.


In [ ]:
!pwd
!ls
import os
os.chdir("pytorch-YOLOv4")
from tool.darknet2pytorch import Darknet

import cv2

/content
drive  pytorch-YOLOv4  sample_data


In [ ]:
m = Darknet('./cfg/yolov4-tiny.cfg')
m.load_weights('../yolov4-tiny.weights')

In [ ]:
m

Darknet(
  (models): ModuleList(
    (0): Sequential(
      (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky1): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Sequential(
      (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky2): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Sequential(
      (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky3): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (3): EmptyModule()
    (4): Sequential(
      (conv4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn4):

In [ ]:
m.models[0]

Sequential(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky1): LeakyReLU(negative_slope=0.1, inplace=True)
)

In [ ]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Code for fusing - Our model starts here

In [ ]:
os.mkdir('../Layer1')

In [ ]:
write_layer_outputs_to_file = True
write_layer_params_to_file  = True

features = {}
def get_features(name):
  def hook(model, input, output):
    features[name] = output.detach()
  return hook

model = m.models[0] #Change the model numbers here. This is the first layer now
model.eval()
print(model)

inp = torch.ones(1, 3, 416, 416)

raw_layers = []
for layer in model.named_modules():
  raw_layers.append(layer[0])

leaf_layers = []
for i in range(1, len(raw_layers)-1):
  curr_layer = raw_layers[i]
  next_layer = raw_layers[i+1]
  if(next_layer[:len(curr_layer)+1] != curr_layer + "."):
    leaf_layers.append(curr_layer)
leaf_layers.append(next_layer)

layers = []
for i in range(len(leaf_layers)):
  layers.append(re.sub(r"\.(\d)",r"[\1]",leaf_layers[i]))

for i in range(len(layers)):
  layer = layers[i]
  layer_hook = "model." + layer + ".register_forward_hook(get_features('" + layer + "'))"
  exec(layer_hook)

# Run inference
outp = model(inp)

EPS = 10 ** -5 # constant

if(write_layer_outputs_to_file):
  # Write layer outputs
  for i in range(len(layers)):
    layer = layers[i]
    if(layer in features.keys()):
      layer_name = layer.replace("].","_")
      layer_name = layer_name.replace("[", "_")
      layer_name = layer_name.replace("]", "")
      filename = "../" +  layer_name + ".bin"
      with open(filename,"wb") as f:
        features[layer].cpu().numpy().tofile(f)
      print("Layer " + str(i) + " feature map printed to " + filename)

if(write_layer_params_to_file):
  # Write layer params
  for i in range(len(layers)):
    layer = layers[i]
    if('conv' in layer or 'downsample[0]' in layer):
      conv_layer_name = layer.replace("].","_")
      conv_layer_name = conv_layer_name.replace("[", "_")
      conv_layer_name = conv_layer_name.replace("]", "")
      
      conv_param_name = layer.replace("[",".")
      conv_param_name = conv_param_name.replace("]","")
      
      conv_weight = model.state_dict()[conv_param_name+'.weight']

    if('bn' in layer or 'downsample[1]' in layer):
      bn_layer_name = layer.replace("].","_")
      bn_layer_name = bn_layer_name.replace("[", "_")
      bn_layer_name = bn_layer_name.replace("]", "")
      
      bn_param_name = layer.replace("[",".")
      bn_param_name = bn_param_name.replace("]","")
      
      bn_weight = model.state_dict()[bn_param_name+'.weight']
      bn_bias   = model.state_dict()[bn_param_name+'.bias']
      bn_mean   = model.state_dict()[bn_param_name+'.running_mean']
      bn_var    = model.state_dict()[bn_param_name+'.running_var']

      bn_factor    = torch.div(bn_weight,torch.sqrt(bn_var+EPS)).view(-1,1,1,1)
      fused_weight = torch.mul(conv_weight, bn_factor)
      fused_bias   = bn_bias - torch.div(torch.mul(bn_weight,bn_mean),torch.sqrt(bn_var+EPS))

      if('downsample' in bn_layer_name):
        layer_number = '0'
        layer_prefix = bn_layer_name[0:bn_layer_name.find('downsample')]
      else:
        layer_number = conv_layer_name[-1]
        layer_prefix = bn_layer_name[0:bn_layer_name.find('bn')]
      
      weights_filename = "../Layer1/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_weights.bin" #This gives the bin for weights
      bias_filename    = "../Layer1/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_bias.bin" #This gives the bin for bias

      with open(weights_filename, "wb") as f:
        fused_weight.detach().numpy().tofile(f)
      print("Fused weights of " + layer_prefix + layer_number + " printed to file " + weights_filename)

      with open(bias_filename, "wb") as f:
        fused_bias.detach().numpy().tofile(f)
      print("Fused biases of " + layer_prefix + layer_number + " printed to file " + bias_filename)



fusedconv = torch.nn.Conv2d(
    model.conv1.in_channels,
    model.conv1.out_channels,
    kernel_size=model.conv1.kernel_size,
    stride=model.conv1.stride,
    padding=model.conv1.padding,
    bias=True
    )

weightfile1 = '../Layer1/fused_conv1_bn1_bias.bin'

import numpy as np

fp = open(weightfile1, 'rb')
buf = np.fromfile(fp, dtype=np.float32)
fp.close()

bias = buf

weightfile2 = '../Layer1/fused_conv1_bn1_weights.bin'

fp = open(weightfile2, 'rb')
buf1 = np.fromfile(fp, dtype=np.float32)
fp.close()

weight = buf1.reshape(fusedconv.weight.data.shape)
print(weight.shape)

torch.set_grad_enabled(False)

fusedconv.weight.copy_(torch.from_numpy(weight))
fusedconv.bias.copy_(torch.from_numpy(bias))

new_model = torch.nn.Sequential()
new_model.append(fusedconv)
new_model.append(model.leaky1)


img = cv2.imread('data/dog.jpg')
sized = cv2.resize(img, (m.width, m.height))
sized = cv2.cvtColor(sized, cv2.COLOR_BGR2RGB)
img = torch.from_numpy(sized.transpose(2, 0, 1)).float().div(255.0).unsqueeze(0)

y_original = model(img)
y_original
y_new = new_model(img)

d = torch.mean(torch.pow(y_original - y_new,2))
print("error: %.15f" % d)

img_filename = '../Layer1/conv_layer1_input.bin'

with open(img_filename, "wb") as f:
        img.detach().numpy().tofile(f)
print("Image input stored in the file" + img_filename)

out_filename = '../Layer1/conv_layer1_output.bin'

with open(out_filename, "wb") as f:
        y_new.detach().numpy().tofile(f)
print("Conv layer1 output stored in the file" + out_filename)



Sequential(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky1): LeakyReLU(negative_slope=0.1, inplace=True)
)
Layer 0 feature map printed to ../conv1.bin
Layer 1 feature map printed to ../bn1.bin
Layer 2 feature map printed to ../leaky1.bin
Fused weights of 1 printed to file ../Layer1/fused_conv1_bn1_weights.bin
Fused biases of 1 printed to file ../Layer1/fused_conv1_bn1_bias.bin
(32, 3, 3, 3)
error: 0.000000000000659
Image input stored in the file../Layer1/conv_layer1_input.bin
Conv layer1 output stored in the file../Layer1/conv_layer1_output.bin


In [ ]:
import time

start_time = time.time()

y__original = model(img)

end_time = time.time()

time_spent = end_time - start_time

print(time_spent)

0.003611326217651367


# Code to convert the bias and weights to torch params and copy to the model

# Evaluation

# Layer 2 Convolution BN ReLU

In [ ]:
os.mkdir('../Layer2')

In [ ]:
write_layer_outputs_to_file = True
write_layer_params_to_file  = True

features = {}
def get_features(name):
  def hook(model, input, output):
    features[name] = output.detach()
  return hook

model = m.models[1] #Change the model numbers here. This is the first layer now
model.eval()
print(model)



inp = torch.ones(1, 32, 208, 208)

raw_layers = []
for layer in model.named_modules():
  raw_layers.append(layer[0])

leaf_layers = []
for i in range(1, len(raw_layers)-1):
  curr_layer = raw_layers[i]
  next_layer = raw_layers[i+1]
  if(next_layer[:len(curr_layer)+1] != curr_layer + "."):
    leaf_layers.append(curr_layer)
leaf_layers.append(next_layer)

layers = []
for i in range(len(leaf_layers)):
  layers.append(re.sub(r"\.(\d)",r"[\1]",leaf_layers[i]))

for i in range(len(layers)):
  layer = layers[i]
  layer_hook = "model." + layer + ".register_forward_hook(get_features('" + layer + "'))"
  exec(layer_hook)

# Run inference
outp = model(inp)

EPS = 10 ** -5 # constant

if(write_layer_outputs_to_file):
  # Write layer outputs
  for i in range(len(layers)):
    layer = layers[i]
    if(layer in features.keys()):
      layer_name = layer.replace("].","_")
      layer_name = layer_name.replace("[", "_")
      layer_name = layer_name.replace("]", "")
      filename = "../" +  layer_name + ".bin"
      with open(filename,"wb") as f:
        features[layer].cpu().numpy().tofile(f)
      print("Layer " + str(i) + " feature map printed to " + filename)

if(write_layer_params_to_file):
  # Write layer params
  for i in range(len(layers)):
    layer = layers[i]
    if('conv' in layer or 'downsample[0]' in layer):
      conv_layer_name = layer.replace("].","_")
      conv_layer_name = conv_layer_name.replace("[", "_")
      conv_layer_name = conv_layer_name.replace("]", "")
      
      conv_param_name = layer.replace("[",".")
      conv_param_name = conv_param_name.replace("]","")
      
      conv_weight = model.state_dict()[conv_param_name+'.weight']

    if('bn' in layer or 'downsample[1]' in layer):
      bn_layer_name = layer.replace("].","_")
      bn_layer_name = bn_layer_name.replace("[", "_")
      bn_layer_name = bn_layer_name.replace("]", "")
      
      bn_param_name = layer.replace("[",".")
      bn_param_name = bn_param_name.replace("]","")
      
      bn_weight = model.state_dict()[bn_param_name+'.weight']
      bn_bias   = model.state_dict()[bn_param_name+'.bias']
      bn_mean   = model.state_dict()[bn_param_name+'.running_mean']
      bn_var    = model.state_dict()[bn_param_name+'.running_var']

      bn_factor    = torch.div(bn_weight,torch.sqrt(bn_var+EPS)).view(-1,1,1,1)
      fused_weight = torch.mul(conv_weight, bn_factor)
      fused_bias   = bn_bias - torch.div(torch.mul(bn_weight,bn_mean),torch.sqrt(bn_var+EPS))

      if('downsample' in bn_layer_name):
        layer_number = '0'
        layer_prefix = bn_layer_name[0:bn_layer_name.find('downsample')]
      else:
        layer_number = conv_layer_name[-1]
        layer_prefix = bn_layer_name[0:bn_layer_name.find('bn')]
      
      weights_filename = "../Layer2/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_weights.bin" #This gives the bin for weights
      bias_filename    = "../Layer2/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_bias.bin" #This gives the bin for bias

      with open(weights_filename, "wb") as f:
        fused_weight.detach().numpy().tofile(f)
      print("Fused weights of " + layer_prefix + layer_number + " printed to file " + weights_filename)

      with open(bias_filename, "wb") as f:
        fused_bias.detach().numpy().tofile(f)
      print("Fused biases of " + layer_prefix + layer_number + " printed to file " + bias_filename)



fusedconv2 = torch.nn.Conv2d(
    model.conv2.in_channels,
    model.conv2.out_channels,
    kernel_size=model.conv2.kernel_size,
    stride=model.conv2.stride,
    padding=model.conv2.padding,
    bias=True
    )

weightfile1 = '../Layer2/fused_conv2_bn2_bias.bin'

import numpy as np

fp = open(weightfile1, 'rb')
buf = np.fromfile(fp, dtype=np.float32)
fp.close()

bias = buf

weightfile2 = '../Layer2/fused_conv2_bn2_weights.bin'

fp = open(weightfile2, 'rb')
buf1 = np.fromfile(fp, dtype=np.float32)
fp.close()


weight = buf1.reshape(fusedconv2.weight.data.shape)
print(weight.shape)

torch.set_grad_enabled(False)

fusedconv2.weight.copy_(torch.from_numpy(weight))
fusedconv2.bias.copy_(torch.from_numpy(bias))

new_model = torch.nn.Sequential()
new_model.append(fusedconv2)
new_model.append(model.leaky2)

inputfile = '../Layer1/conv_layer1_output.bin'

fp1 = open(inputfile,'rb')
buf2 = np.fromfile(fp1,dtype=np.float32)
fp1.close()

input = buf2.reshape((1,32,208,208))
input2 = torch.from_numpy(input)


y_input_layer2 = m.models[0](img)

print(y_input_layer2.shape)

y_original = model(y_input_layer2)

y_new = new_model(y_input_layer2)

d = torch.mean(torch.pow(y_original - y_new,2))
print("error: %.15f" % d)

img_filename = '../Layer2/conv_layer2_input.bin'

with open(img_filename, "wb") as f:
        img.detach().numpy().tofile(f)
print("Image input stored in the file" + img_filename)

out_filename = '../Layer2/conv_layer2_output.bin'

with open(out_filename, "wb") as f:
        y_new.detach().numpy().tofile(f)
print("Conv layer2 output stored in the file" + out_filename)


Sequential(
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky2): LeakyReLU(negative_slope=0.1, inplace=True)
)
Layer 0 feature map printed to ../conv2.bin
Layer 1 feature map printed to ../bn2.bin
Layer 2 feature map printed to ../leaky2.bin
Fused weights of 2 printed to file ../Layer2/fused_conv2_bn2_weights.bin
Fused biases of 2 printed to file ../Layer2/fused_conv2_bn2_bias.bin
(64, 32, 3, 3)
torch.Size([1, 32, 208, 208])
error: 0.000000000000755
Image input stored in the file../Layer2/conv_layer2_input.bin
Conv layer2 output stored in the file../Layer2/conv_layer2_output.bin


In [ ]:
with open('../relu_output_2.bin', 'rb') as f:
    y_new_other = np.reshape(np.fromfile(f,dtype=np.float32),(1,64,104,104))
    
y__new = torch.from_numpy(y_new_other)

In [ ]:

d = torch.mean(torch.pow(y__new - y_original,2))
print("error: %.15f" % d)

error: 0.133518382906914


Layer - 3

In [ ]:
os.mkdir('../Layer3')

In [ ]:
layer3_model = m.models[2]

write_layer_outputs_to_file = True
write_layer_params_to_file  = True

features = {}
def get_features(name):
  def hook(model, input, output):
    features[name] = output.detach()
  return hook

model = layer3_model #Change the model numbers here. This is the first layer now
model.eval()
print(model)

inp = torch.ones(1, 64, 104, 104)

raw_layers = []
for layer in model.named_modules():
  raw_layers.append(layer[0])

leaf_layers = []
for i in range(1, len(raw_layers)-1):
  curr_layer = raw_layers[i]
  next_layer = raw_layers[i+1]
  if(next_layer[:len(curr_layer)+1] != curr_layer + "."):
    leaf_layers.append(curr_layer)
leaf_layers.append(next_layer)

layers = []
for i in range(len(leaf_layers)):
  layers.append(re.sub(r"\.(\d)",r"[\1]",leaf_layers[i]))

for i in range(len(layers)):
  layer = layers[i]
  layer_hook = "model." + layer + ".register_forward_hook(get_features('" + layer + "'))"
  exec(layer_hook)

# Run inference
outp = model(inp)

EPS = 10 ** -5 # constant

if(write_layer_outputs_to_file):
  # Write layer outputs
  for i in range(len(layers)):
    layer = layers[i]
    if(layer in features.keys()):
      layer_name = layer.replace("].","_")
      layer_name = layer_name.replace("[", "_")
      layer_name = layer_name.replace("]", "")
      filename = "../" +  layer_name + ".bin"
      with open(filename,"wb") as f:
        features[layer].cpu().numpy().tofile(f)
      print("Layer " + str(i) + " feature map printed to " + filename)

if(write_layer_params_to_file):
  # Write layer params
  for i in range(len(layers)):
    layer = layers[i]
    if('conv' in layer or 'downsample[0]' in layer):
      conv_layer_name = layer.replace("].","_")
      conv_layer_name = conv_layer_name.replace("[", "_")
      conv_layer_name = conv_layer_name.replace("]", "")
      
      conv_param_name = layer.replace("[",".")
      conv_param_name = conv_param_name.replace("]","")
      
      conv_weight = model.state_dict()[conv_param_name+'.weight']

    if('bn' in layer or 'downsample[1]' in layer):
      bn_layer_name = layer.replace("].","_")
      bn_layer_name = bn_layer_name.replace("[", "_")
      bn_layer_name = bn_layer_name.replace("]", "")
      
      bn_param_name = layer.replace("[",".")
      bn_param_name = bn_param_name.replace("]","")
      
      bn_weight = model.state_dict()[bn_param_name+'.weight']
      bn_bias   = model.state_dict()[bn_param_name+'.bias']
      bn_mean   = model.state_dict()[bn_param_name+'.running_mean']
      bn_var    = model.state_dict()[bn_param_name+'.running_var']

      bn_factor    = torch.div(bn_weight,torch.sqrt(bn_var+EPS)).view(-1,1,1,1)
      fused_weight = torch.mul(conv_weight, bn_factor)
      fused_bias   = bn_bias - torch.div(torch.mul(bn_weight,bn_mean),torch.sqrt(bn_var+EPS))

      if('downsample' in bn_layer_name):
        layer_number = '0'
        layer_prefix = bn_layer_name[0:bn_layer_name.find('downsample')]
      else:
        layer_number = conv_layer_name[-1]
        layer_prefix = bn_layer_name[0:bn_layer_name.find('bn')]

      weights_filename = "../Layer3/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_weights.bin" #This gives the bin for weights
      bias_filename    = "../Layer3/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_bias.bin" #This gives the bin for bias

      with open(weights_filename, "wb") as f:
        fused_weight.detach().numpy().tofile(f)
      print("Fused weights of " + layer_prefix + layer_number + " printed to file " + weights_filename)

      with open(bias_filename, "wb") as f:
        fused_bias.detach().numpy().tofile(f)
      print("Fused biases of " + layer_prefix + layer_number + " printed to file " + bias_filename)



fusedconv3 = torch.nn.Conv2d(
    model.conv3.in_channels,
    model.conv3.out_channels,
    kernel_size=model.conv3.kernel_size,
    stride=model.conv3.stride,
    padding=model.conv3.padding,
    bias=True
    )

weightfile1 = '../Layer3/fused_conv3_bn3_bias.bin'

import numpy as np

fp = open(weightfile1, 'rb')
buf = np.fromfile(fp, dtype=np.float32)
fp.close()

bias = buf

weightfile2 = '../Layer3/fused_conv3_bn3_weights.bin'

fp = open(weightfile2, 'rb')
buf1 = np.fromfile(fp, dtype=np.float32)
fp.close()

weight = buf1.reshape(fusedconv3.weight.data.shape)
print(weight.shape)

torch.set_grad_enabled(False)

fusedconv3.weight.copy_(torch.from_numpy(weight))
fusedconv3.bias.copy_(torch.from_numpy(bias))

new_model3 = torch.nn.Sequential()
new_model3.append(fusedconv3)
new_model3.append(model.leaky3)


img = cv2.imread('data/dog.jpg')
sized = cv2.resize(img, (m.width, m.height))
sized = cv2.cvtColor(sized, cv2.COLOR_BGR2RGB)
img = torch.from_numpy(sized.transpose(2, 0, 1)).float().div(255.0).unsqueeze(0)

y1 = m.models[0](img)
y2 = m.models[1](y1)
y3 = m.models[2](y2)

y_new3 = new_model3(y2)

d = torch.mean(torch.pow(y3 - y_new3,2))
print("error: %.15f" % d)

layer3_input_filename = '../Layer3/conv_layer3_input.bin'

with open(layer3_input_filename, "wb") as f:
        y2.detach().numpy().tofile(f)
print("Image input stored in the file" + layer3_input_filename)

out_filename = '../Layer3/conv_layer3_output.bin'

with open(out_filename, "wb") as f:
        y_new3.detach().numpy().tofile(f)
print("Conv layer2 output stored in the file" + out_filename)


Sequential(
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky3): LeakyReLU(negative_slope=0.1, inplace=True)
)
Layer 0 feature map printed to ../conv3.bin
Layer 1 feature map printed to ../bn3.bin
Layer 2 feature map printed to ../leaky3.bin
Fused weights of 3 printed to file ../Layer3/fused_conv3_bn3_weights.bin
Fused biases of 3 printed to file ../Layer3/fused_conv3_bn3_bias.bin
(64, 64, 3, 3)
error: 0.000000000001471
Image input stored in the file../Layer3/conv_layer3_input.bin
Conv layer2 output stored in the file../Layer3/conv_layer3_output.bin


In [ ]:
os.mkdir('../Layer4')

In [ ]:
m.models[3]

EmptyModule()

In [ ]:
y4 = m.models[3](y3)

In [ ]:
y4.shape

torch.Size([1, 64, 104, 104])

In [ ]:
y5 = m.models[4](y4[:,32:64,:,:])

In [ ]:
os.mkdir('../Layer4')

# Conv Layer 4

In [ ]:
layer4_model = m.models[4]

write_layer_outputs_to_file = True
write_layer_params_to_file  = True

features = {}
def get_features(name):
  def hook(model, input, output):
    features[name] = output.detach()
  return hook

model = layer4_model #Change the model numbers here. This is the first layer now
model.eval()
print(model)

inp = torch.ones(1, 32, 104, 104)

raw_layers = []
for layer in model.named_modules():
  raw_layers.append(layer[0])

leaf_layers = []
for i in range(1, len(raw_layers)-1):
  curr_layer = raw_layers[i]
  next_layer = raw_layers[i+1]
  if(next_layer[:len(curr_layer)+1] != curr_layer + "."):
    leaf_layers.append(curr_layer)
leaf_layers.append(next_layer)

layers = []
for i in range(len(leaf_layers)):
  layers.append(re.sub(r"\.(\d)",r"[\1]",leaf_layers[i]))

for i in range(len(layers)):
  layer = layers[i]
  layer_hook = "model." + layer + ".register_forward_hook(get_features('" + layer + "'))"
  exec(layer_hook)

# Run inference
outp = model(inp)

EPS = 10 ** -5 # constant

if(write_layer_outputs_to_file):
  # Write layer outputs
  for i in range(len(layers)):
    layer = layers[i]
    if(layer in features.keys()):
      layer_name = layer.replace("].","_")
      layer_name = layer_name.replace("[", "_")
      layer_name = layer_name.replace("]", "")
      filename = "../" +  layer_name + ".bin"
      with open(filename,"wb") as f:
        features[layer].cpu().numpy().tofile(f)
      print("Layer " + str(i) + " feature map printed to " + filename)

if(write_layer_params_to_file):
  # Write layer params
  for i in range(len(layers)):
    layer = layers[i]
    if('conv' in layer or 'downsample[0]' in layer):
      conv_layer_name = layer.replace("].","_")
      conv_layer_name = conv_layer_name.replace("[", "_")
      conv_layer_name = conv_layer_name.replace("]", "")
      
      conv_param_name = layer.replace("[",".")
      conv_param_name = conv_param_name.replace("]","")
      
      conv_weight = model.state_dict()[conv_param_name+'.weight']

    if('bn' in layer or 'downsample[1]' in layer):
      bn_layer_name = layer.replace("].","_")
      bn_layer_name = bn_layer_name.replace("[", "_")
      bn_layer_name = bn_layer_name.replace("]", "")
      
      bn_param_name = layer.replace("[",".")
      bn_param_name = bn_param_name.replace("]","")
      
      bn_weight = model.state_dict()[bn_param_name+'.weight']
      bn_bias   = model.state_dict()[bn_param_name+'.bias']
      bn_mean   = model.state_dict()[bn_param_name+'.running_mean']
      bn_var    = model.state_dict()[bn_param_name+'.running_var']

      bn_factor    = torch.div(bn_weight,torch.sqrt(bn_var+EPS)).view(-1,1,1,1)
      fused_weight = torch.mul(conv_weight, bn_factor)
      fused_bias   = bn_bias - torch.div(torch.mul(bn_weight,bn_mean),torch.sqrt(bn_var+EPS))

      if('downsample' in bn_layer_name):
        layer_number = '0'
        layer_prefix = bn_layer_name[0:bn_layer_name.find('downsample')]
      else:
        layer_number = conv_layer_name[-1]
        layer_prefix = bn_layer_name[0:bn_layer_name.find('bn')]

      

      weights_filename = "../Layer4/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_weights.bin" #This gives the bin for weights
      bias_filename    = "../Layer4/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_bias.bin" #This gives the bin for bias

      with open(weights_filename, "wb") as f:
        fused_weight.detach().numpy().tofile(f)
      print("Fused weights of " + layer_prefix + layer_number + " printed to file " + weights_filename)

      with open(bias_filename, "wb") as f:
        fused_bias.detach().numpy().tofile(f)
      print("Fused biases of " + layer_prefix + layer_number + " printed to file " + bias_filename)



fusedconv4 = torch.nn.Conv2d(
    model.conv4.in_channels,
    model.conv4.out_channels,
    kernel_size=model.conv4.kernel_size,
    stride=model.conv4.stride,
    padding=model.conv4.padding,
    bias=True
    )

weightfile1 = '../Layer4/fused_conv4_bn4_bias.bin'

import numpy as np

fp = open(weightfile1, 'rb')
buf = np.fromfile(fp, dtype=np.float32)
fp.close()

bias = buf

weightfile2 = '../Layer4/fused_conv4_bn4_weights.bin'

fp = open(weightfile2, 'rb')
buf1 = np.fromfile(fp, dtype=np.float32)
fp.close()

weight = buf1.reshape(fusedconv4.weight.data.shape)
print(weight.shape)

torch.set_grad_enabled(False)

fusedconv4.weight.copy_(torch.from_numpy(weight))
fusedconv4.bias.copy_(torch.from_numpy(bias))

new_model4 = torch.nn.Sequential()
new_model4.append(fusedconv4)
new_model4.append(model.leaky4)


img = cv2.imread('data/dog.jpg')
sized = cv2.resize(img, (m.width, m.height))
sized = cv2.cvtColor(sized, cv2.COLOR_BGR2RGB)
img = torch.from_numpy(sized.transpose(2, 0, 1)).float().div(255.0).unsqueeze(0)

y1 = m.models[0](img)
y2 = m.models[1](y1)
y3 = m.models[2](y2)
y4 = m.models[3](y3)
y5 = m.models[4](y4[:,32:64,:,:])

y_new5 = new_model4(y4[:,32:64,:,:])

d = torch.mean(torch.pow(y5 - y_new5,2))
print("error: %.15f" % d)


y5_1 = m.models[4](y4[:,0:32,:,:])
y5_new_1 = new_model4(y4[:,0:32,:,:])

d1 =torch.mean(torch.pow(y5_1 - y5_new_1,2))
print("error: %.15f" % d1)
 

layer4_input_filename = '../Layer4/conv_layer4_input.bin'

with open(layer4_input_filename, "wb") as f:
        y4.detach().numpy().tofile(f)
print("Image input stored in the file" + layer4_input_filename)

y4_ = y4[:,32:64,:,:]

layer4_input_2_filename = '../Layer4/conv_layer4_inputs.bin'
with open(layer4_input_2_filename, "wb") as f:
        y4_.detach().numpy().tofile(f)
print("Image input stored in the file" + layer4_input_filename)


out_filename = '../Layer4/conv_layer4_output.bin'

with open(out_filename, "wb") as f:
        y_new5.detach().numpy().tofile(f)
print("Conv layer2 output stored in the file" + out_filename)

layer5_in_filename = '../Layer4/conv_layer5_input.bin'

with open(layer5_in_filename, "wb") as f:
        y_new5.detach().numpy().tofile(f)
print("Conv layer5 input stored in the file" + layer5_in_filename)

Sequential(
  (conv4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky4): LeakyReLU(negative_slope=0.1, inplace=True)
)
Layer 0 feature map printed to ../conv4.bin
Layer 1 feature map printed to ../bn4.bin
Layer 2 feature map printed to ../leaky4.bin
Fused weights of 4 printed to file ../Layer4/fused_conv4_bn4_weights.bin
Fused biases of 4 printed to file ../Layer4/fused_conv4_bn4_bias.bin
(32, 32, 3, 3)
error: 0.000000000001077
error: 0.000000000001313
Image input stored in the file../Layer4/conv_layer4_input.bin
Image input stored in the file../Layer4/conv_layer4_input.bin
Conv layer2 output stored in the file../Layer4/conv_layer4_output.bin
Conv layer5 input stored in the file../Layer4/conv_layer5_input.bin


In [ ]:
os.mkdir('../Layer5')

In [ ]:
layer5_model = m.models[5]

write_layer_outputs_to_file = True
write_layer_params_to_file  = True

features = {}
def get_features(name):
  def hook(model, input, output):
    features[name] = output.detach()
  return hook

model = layer5_model #Change the model numbers here. This is the first layer now
model.eval()
print(model)

inp = torch.ones(1, 32, 104, 104)

raw_layers = []
for layer in model.named_modules():
  raw_layers.append(layer[0])

leaf_layers = []
for i in range(1, len(raw_layers)-1):
  curr_layer = raw_layers[i]
  next_layer = raw_layers[i+1]
  if(next_layer[:len(curr_layer)+1] != curr_layer + "."):
    leaf_layers.append(curr_layer)
leaf_layers.append(next_layer)

layers = []
for i in range(len(leaf_layers)):
  layers.append(re.sub(r"\.(\d)",r"[\1]",leaf_layers[i]))

for i in range(len(layers)):
  layer = layers[i]
  layer_hook = "model." + layer + ".register_forward_hook(get_features('" + layer + "'))"
  exec(layer_hook)

# Run inference
outp = model(inp)

EPS = 10 ** -5 # constant

if(write_layer_outputs_to_file):
  # Write layer outputs
  for i in range(len(layers)):
    layer = layers[i]
    if(layer in features.keys()):
      layer_name = layer.replace("].","_")
      layer_name = layer_name.replace("[", "_")
      layer_name = layer_name.replace("]", "")
      filename = "../" +  layer_name + ".bin"
      with open(filename,"wb") as f:
        features[layer].cpu().numpy().tofile(f)
      print("Layer " + str(i) + " feature map printed to " + filename)

if(write_layer_params_to_file):
  # Write layer params
  for i in range(len(layers)):
    layer = layers[i]
    if('conv' in layer or 'downsample[0]' in layer):
      conv_layer_name = layer.replace("].","_")
      conv_layer_name = conv_layer_name.replace("[", "_")
      conv_layer_name = conv_layer_name.replace("]", "")
      
      conv_param_name = layer.replace("[",".")
      conv_param_name = conv_param_name.replace("]","")
      
      conv_weight = model.state_dict()[conv_param_name+'.weight']

    if('bn' in layer or 'downsample[1]' in layer):
      bn_layer_name = layer.replace("].","_")
      bn_layer_name = bn_layer_name.replace("[", "_")
      bn_layer_name = bn_layer_name.replace("]", "")
      
      bn_param_name = layer.replace("[",".")
      bn_param_name = bn_param_name.replace("]","")
      
      bn_weight = model.state_dict()[bn_param_name+'.weight']
      bn_bias   = model.state_dict()[bn_param_name+'.bias']
      bn_mean   = model.state_dict()[bn_param_name+'.running_mean']
      bn_var    = model.state_dict()[bn_param_name+'.running_var']

      bn_factor    = torch.div(bn_weight,torch.sqrt(bn_var+EPS)).view(-1,1,1,1)
      fused_weight = torch.mul(conv_weight, bn_factor)
      fused_bias   = bn_bias - torch.div(torch.mul(bn_weight,bn_mean),torch.sqrt(bn_var+EPS))

      if('downsample' in bn_layer_name):
        layer_number = '0'
        layer_prefix = bn_layer_name[0:bn_layer_name.find('downsample')]
      else:
        layer_number = conv_layer_name[-1]
        layer_prefix = bn_layer_name[0:bn_layer_name.find('bn')]

      

      weights_filename = "../Layer5/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_weights.bin" #This gives the bin for weights
      bias_filename    = "../Layer5/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_bias.bin" #This gives the bin for bias

      with open(weights_filename, "wb") as f:
        fused_weight.detach().numpy().tofile(f)
      print("Fused weights of " + layer_prefix + layer_number + " printed to file " + weights_filename)

      with open(bias_filename, "wb") as f:
        fused_bias.detach().numpy().tofile(f)
      print("Fused biases of " + layer_prefix + layer_number + " printed to file " + bias_filename)



fusedconv5 = torch.nn.Conv2d(
    model.conv5.in_channels,
    model.conv5.out_channels,
    kernel_size=model.conv5.kernel_size,
    stride=model.conv5.stride,
    padding=model.conv5.padding,
    bias=True
    )

weightfile1 = '../Layer5/fused_conv5_bn5_bias.bin'

import numpy as np

fp = open(weightfile1, 'rb')
buf = np.fromfile(fp, dtype=np.float32)
fp.close()

bias = buf

weightfile2 = '../Layer5/fused_conv5_bn5_weights.bin'

fp = open(weightfile2, 'rb')
buf1 = np.fromfile(fp, dtype=np.float32)
fp.close()

weight = buf1.reshape(fusedconv5.weight.data.shape)
print(weight.shape)

torch.set_grad_enabled(False)

fusedconv5.weight.copy_(torch.from_numpy(weight))
fusedconv5.bias.copy_(torch.from_numpy(bias))

new_model5 = torch.nn.Sequential()
new_model5.append(fusedconv5)
new_model5.append(model.leaky5)


img = cv2.imread('data/dog.jpg')
sized = cv2.resize(img, (m.width, m.height))
sized = cv2.cvtColor(sized, cv2.COLOR_BGR2RGB)
img = torch.from_numpy(sized.transpose(2, 0, 1)).float().div(255.0).unsqueeze(0)

y1 = m.models[0](img)
y2 = m.models[1](y1)
y3 = m.models[2](y2)
y4 = m.models[3](y3)
y5 = m.models[4](y4[:,32:64,:,:])
y6 = m.models[5](y5)

y_new6 = new_model5(y5)

d = torch.mean(torch.pow(y6 - y_new6,2))
print("error: %.15f" % d)

layer5_input_filename = '../Layer5/conv_layer5_input.bin'

with open(layer5_input_filename, "wb") as f:
        y5.detach().numpy().tofile(f)
print("Image input stored in the file" + layer5_input_filename)

out_filename = '../Layer5/conv_layer5_output.bin'

with open(out_filename, "wb") as f:
        y_new6.detach().numpy().tofile(f)
print("Conv layer output stored in the file" + out_filename)

layer5_in_filename = '../Layer5/conv_layer6_input.bin'

with open(layer5_in_filename, "wb") as f:
        y_new6.detach().numpy().tofile(f)
print("Conv layer6 input stored in the file" + layer5_in_filename)

Sequential(
  (conv5): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky5): LeakyReLU(negative_slope=0.1, inplace=True)
)
Layer 0 feature map printed to ../conv5.bin
Layer 1 feature map printed to ../bn5.bin
Layer 2 feature map printed to ../leaky5.bin
Fused weights of 5 printed to file ../Layer5/fused_conv5_bn5_weights.bin
Fused biases of 5 printed to file ../Layer5/fused_conv5_bn5_bias.bin
(32, 32, 3, 3)
error: 0.000000000001383
Image input stored in the file../Layer5/conv_layer5_input.bin
Conv layer output stored in the file../Layer5/conv_layer5_output.bin
Conv layer6 input stored in the file../Layer5/conv_layer6_input.bin


In [ ]:
layer6_input = torch.cat((y6,y5),axis=1)

In [ ]:
os.mkdir('../Layer6')

FileExistsError: ignored

In [ ]:
layer6_model = m.models[7]

write_layer_outputs_to_file = True
write_layer_params_to_file  = True

features = {}
def get_features(name):
  def hook(model, input, output):
    features[name] = output.detach()
  return hook

model = layer6_model #Change the model numbers here. This is the first layer now
model.eval()
print(model)

inp = torch.ones(1, 64, 104, 104)

raw_layers = []
for layer in model.named_modules():
  raw_layers.append(layer[0])

leaf_layers = []
for i in range(1, len(raw_layers)-1):
  curr_layer = raw_layers[i]
  next_layer = raw_layers[i+1]
  if(next_layer[:len(curr_layer)+1] != curr_layer + "."):
    leaf_layers.append(curr_layer)
leaf_layers.append(next_layer)

layers = []
for i in range(len(leaf_layers)):
  layers.append(re.sub(r"\.(\d)",r"[\1]",leaf_layers[i]))

for i in range(len(layers)):
  layer = layers[i]
  layer_hook = "model." + layer + ".register_forward_hook(get_features('" + layer + "'))"
  exec(layer_hook)

# Run inference
outp = model(inp)

EPS = 10 ** -5 # constant

if(write_layer_outputs_to_file):
  # Write layer outputs
  for i in range(len(layers)):
    layer = layers[i]
    if(layer in features.keys()):
      layer_name = layer.replace("].","_")
      layer_name = layer_name.replace("[", "_")
      layer_name = layer_name.replace("]", "")
      filename = "../" +  layer_name + ".bin"
      with open(filename,"wb") as f:
        features[layer].cpu().numpy().tofile(f)
      print("Layer " + str(i) + " feature map printed to " + filename)

if(write_layer_params_to_file):
  # Write layer params
  for i in range(len(layers)):
    layer = layers[i]
    if('conv' in layer or 'downsample[0]' in layer):
      conv_layer_name = layer.replace("].","_")
      conv_layer_name = conv_layer_name.replace("[", "_")
      conv_layer_name = conv_layer_name.replace("]", "")
      
      conv_param_name = layer.replace("[",".")
      conv_param_name = conv_param_name.replace("]","")
      
      conv_weight = model.state_dict()[conv_param_name+'.weight']

    if('bn' in layer or 'downsample[1]' in layer):
      bn_layer_name = layer.replace("].","_")
      bn_layer_name = bn_layer_name.replace("[", "_")
      bn_layer_name = bn_layer_name.replace("]", "")
      
      bn_param_name = layer.replace("[",".")
      bn_param_name = bn_param_name.replace("]","")
      
      bn_weight = model.state_dict()[bn_param_name+'.weight']
      bn_bias   = model.state_dict()[bn_param_name+'.bias']
      bn_mean   = model.state_dict()[bn_param_name+'.running_mean']
      bn_var    = model.state_dict()[bn_param_name+'.running_var']

      bn_factor    = torch.div(bn_weight,torch.sqrt(bn_var+EPS)).view(-1,1,1,1)
      fused_weight = torch.mul(conv_weight, bn_factor)
      fused_bias   = bn_bias - torch.div(torch.mul(bn_weight,bn_mean),torch.sqrt(bn_var+EPS))

      if('downsample' in bn_layer_name):
        layer_number = '0'
        layer_prefix = bn_layer_name[0:bn_layer_name.find('downsample')]
      else:
        layer_number = conv_layer_name[-1]
        layer_prefix = bn_layer_name[0:bn_layer_name.find('bn')]

      

      weights_filename = "../Layer6/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_weights.bin" #This gives the bin for weights
      bias_filename    = "../Layer6/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_bias.bin" #This gives the bin for bias

      with open(weights_filename, "wb") as f:
        fused_weight.detach().numpy().tofile(f)
      print("Fused weights of " + layer_prefix + layer_number + " printed to file " + weights_filename)

      with open(bias_filename, "wb") as f:
        fused_bias.detach().numpy().tofile(f)
      print("Fused biases of " + layer_prefix + layer_number + " printed to file " + bias_filename)



fusedconv6 = torch.nn.Conv2d(
    model.conv6.in_channels,
    model.conv6.out_channels,
    kernel_size=model.conv6.kernel_size,
    stride=model.conv6.stride,
    padding=model.conv6.padding,
    bias=True
    )

weightfile1 = '../Layer6/fused_conv6_bn6_bias.bin'

import numpy as np

fp = open(weightfile1, 'rb')
buf = np.fromfile(fp, dtype=np.float32)
fp.close()

bias = buf

weightfile2 = '../Layer6/fused_conv6_bn6_weights.bin'

fp = open(weightfile2, 'rb')
buf1 = np.fromfile(fp, dtype=np.float32)
fp.close()

weight = buf1.reshape(fusedconv6.weight.data.shape)
print(weight.shape)

torch.set_grad_enabled(False)

fusedconv6.weight.copy_(torch.from_numpy(weight))
fusedconv6.bias.copy_(torch.from_numpy(bias))

new_model6 = torch.nn.Sequential()
new_model6.append(fusedconv6)
new_model6.append(model.leaky6)


img = cv2.imread('data/dog.jpg')
sized = cv2.resize(img, (m.width, m.height))
sized = cv2.cvtColor(sized, cv2.COLOR_BGR2RGB)
img = torch.from_numpy(sized.transpose(2, 0, 1)).float().div(255.0).unsqueeze(0)

y1 = m.models[0](img)
y2 = m.models[1](y1)
y3 = m.models[2](y2)
y4 = m.models[3](y3)
y5 = m.models[4](y4[:,32:64,:,:])
y6 = m.models[5](y5)
layer6_input = torch.cat((y6,y5),axis=1)
y7 = m.models[7](layer6_input)

y_new7 = new_model6(layer6_input)

d = torch.mean(torch.pow(y7 - y_new7,2))
print("error: %.15f" % d)

layer6_input_filename = '../Layer6/conv_layer6_input.bin'

with open(layer6_input_filename, "wb") as f:
        layer6_input.detach().numpy().tofile(f)
print("Image input stored in the file" + layer6_input_filename)

out_filename = '../Layer6/conv_layer6_output.bin'

with open(out_filename, "wb") as f:
        y_new7.detach().numpy().tofile(f)
print("Conv layer output stored in the file" + out_filename)

layer7_in_filename = '../Layer6/conv_layer7_input.bin'

with open(layer7_in_filename, "wb") as f:
        y_new7.detach().numpy().tofile(f)
print("Conv layer7 input stored in the file" + layer7_in_filename)

Sequential(
  (conv6): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky6): LeakyReLU(negative_slope=0.1, inplace=True)
)
Layer 0 feature map printed to ../conv6.bin
Layer 1 feature map printed to ../bn6.bin
Layer 2 feature map printed to ../leaky6.bin
Fused weights of 6 printed to file ../Layer6/fused_conv6_bn6_weights.bin
Fused biases of 6 printed to file ../Layer6/fused_conv6_bn6_bias.bin
(64, 64, 1, 1)
error: 0.000000000000155
Image input stored in the file../Layer6/conv_layer6_input.bin
Conv layer output stored in the file../Layer6/conv_layer6_output.bin
Conv layer7 input stored in the file../Layer6/conv_layer7_input.bin


In [ ]:
layer7_input = torch.cat((y_new7,y_new3),axis=1)

In [ ]:
layer7_input.shape

torch.Size([1, 128, 104, 104])

In [ ]:
m.models[9]

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

In [ ]:
y9 = m.models[9](layer7_input)

In [ ]:
layer7_input

tensor([[[[ 7.3152e-01, -4.9125e-02, -2.2711e-01,  ..., -2.9255e-02,
           -3.1526e-02,  5.3563e+00],
          [-1.5385e-02, -2.5106e-02, -2.6858e-01,  ..., -5.2941e-02,
           -4.2126e-01, -1.2941e-01],
          [ 1.1590e+00,  3.1158e-01, -9.0390e-02,  ...,  1.5182e+00,
           -2.3955e-01, -3.0923e-01],
          ...,
          [-1.3615e-02,  2.0874e-01,  1.6401e-01,  ..., -7.4391e-02,
           -3.6979e-02, -9.3041e-02],
          [ 8.7522e-01,  8.9358e-01,  1.3700e+00,  ...,  1.6249e-01,
           -8.2644e-02, -5.7963e-02],
          [ 3.5337e+00,  2.6536e+00,  2.4756e+00,  ...,  2.0903e+00,
            6.6415e-01, -6.5191e-02]],

         [[-1.6508e-02,  1.9037e+00,  3.6852e-01,  ..., -3.2171e-01,
           -9.5726e-03, -2.5075e-01],
          [-1.0944e-01,  1.6691e+00,  4.1774e-01,  ..., -5.3844e-01,
           -2.4751e-01, -4.8693e-02],
          [-2.5944e-01, -6.3128e-02,  2.2200e-01,  ..., -2.7392e-01,
           -6.2902e-01,  3.6915e-01],
          ...,
     

In [ ]:
y9

tensor([[[[ 7.3152e-01, -2.2711e-01,  4.2286e-02,  ...,  4.4751e+00,
            1.4639e+00,  5.3563e+00],
          [ 1.5748e+00, -2.5769e-02, -3.9965e-01,  ..., -1.2106e-01,
            1.5182e+00, -2.3955e-01],
          [ 1.8654e+00,  3.0813e-01, -3.6903e-01,  ..., -2.3271e-02,
           -1.8064e-01, -1.3758e-01],
          ...,
          [ 7.0710e-01,  9.6734e-01, -6.7745e-03,  ...,  5.4169e-01,
            1.6281e-01, -7.6980e-02],
          [ 5.0894e-01,  1.6401e-01, -9.3256e-02,  ...,  3.9575e-01,
            6.0742e-01, -3.6979e-02],
          [ 3.5337e+00,  2.4756e+00,  2.3717e+00,  ...,  2.0329e+00,
            2.0903e+00,  6.6415e-01]],

         [[ 1.9037e+00,  4.1774e-01,  3.8525e+00,  ...,  1.9832e+00,
           -1.1867e-01, -9.5726e-03],
          [-6.3128e-02,  2.2200e-01, -7.9004e-02,  ...,  5.8153e-01,
            3.7357e-02,  6.8406e-01],
          [-6.3146e-02, -2.4866e-02, -2.4733e-01,  ..., -2.6140e-02,
           -2.1162e-01,  1.1614e+00],
          ...,
     

In [ ]:
os.mkdir('../Maxpool')


In [ ]:
layer7_input_filename = '../Maxpool/maxpool_input.bin'

with open(layer7_input_filename, "wb") as f:
        layer7_input.detach().numpy().tofile(f)
print("Image input stored in the file" + layer7_input_filename)

out_filename = '../Maxpool/maxpool_output.bin'

with open(out_filename, "wb") as f:
        y9.detach().numpy().tofile(f)
print("Conv layer output stored in the file" + out_filename)

Image input stored in the file../Maxpool/maxpool_input.bin
Conv layer output stored in the file../Maxpool/maxpool_output.bin
